In [23]:
# ignore warnings that might clutter the output, ensuring that the results are clean and readable
import warnings
warnings.filterwarnings('ignore')

# ensure compatibility with Python 2 for division, print function, and unicode literals, making the code more forward-compatible with Python 3
from __future__ import division, print_function, unicode_literals

import os
import numpy as np
import pandas as pd 
from math import log

from itertools import combinations

from IPython.display import display, HTML

In [46]:
# Get the data 
path = "https://drive.google.com/uc?id=11FNXuP673a1Bt77a-G9SlPEx_njqr8Ex"
df = pd.read_csv(path)

print(f"Total Articles: {len(df)}")
df.head()

Total Articles: 293119


,id,title,url,num_points,num_comments,author,created_at
0,12579008,You have two days to comment if you want stem ...,http://www.regulations.gov/document?D=FDA-2015...,1,0,altstar,9/26/2016 3:26
1,12579005,SQLAR the SQLite Archiver,https://www.sqlite.org/sqlar/doc/trunk/README.md,1,0,blacksqr,9/26/2016 3:24
2,12578997,What if we just printed a flatscreen televisio...,https://medium.com/vanmoof/our-secrets-out-f21...,1,0,pavel_lishin,9/26/2016 3:19
3,12578989,algorithmic music,http://cacm.acm.org/magazines/2011/7/109891-al...,1,0,poindontcare,9/26/2016 3:16
4,12578979,How the Data Vault Enables the Next-Gen Data W...,https://www.talend.com/blog/2016/05/12/talend-...,1,0,markgainor1,9/26/2016 3:14


In [32]:
# Preprocess titles from HN posts

# The preprocessing steps involve converting titles to lowercase, removing punctuation, and splitting the titles into tokens (words).
from string import punctuation
punctrans = str.maketrans(dict.fromkeys(punctuation)) 

def tokenize(title):
    x = title.lower() # Lowercase
    x = x.encode('ascii', 'ignore').decode() # Keep only ascii chars.
    x = x.translate(punctrans) # Remove punctuation
    return x.split() # Return tokenized.

texts_tokenized = hn['title'].apply(tokenize)

# Stopwords (common words like "the", "is", etc.) are removed from the tokenized texts to reduce noise in the data.
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))

for i in range(0, 5):
    for text in texts_tokenized:
        for x in text:
            if x in sw:
                text.remove(x)

print(texts_tokenized[:20])

0     [two, days, comment, want, stem, cells, classi...
1                             [sqlar, sqlite, archiver]
2        [printed, flatscreen, television, side, boxes]
3                                  [algorithmic, music]
4     [data, vault, enables, nextgen, data, warehous...
5                            [saving, hassle, shopping]
6     [macalifa, new, opensource, music, app, uwp, w...
7     [github, theweavrsmacalifa, music, player, wri...
8                     [google, allo, first, impression]
9          [advanced, multimedia, linux, command, line]
10              [ask, hn, tld, use, local, development]
11                                        [muroc, maru]
12                   [companies, make, products, worse]
13                           [tuning, aws, sqs, queues]
14                                    [promise, github]
15                              [joint, rd, ups, downs]
16    [ibm, announces, next, implementation, apples,...
17       [amazons, algorithms, dont, find, best,

In [33]:
# Compute unigram and bigram counts

# Unigrams (single words) and bigrams (pairs of words) are counted for calculating word frequencies and co-occurrences necessary for computing PMI scores later.
from collections import Counter
cx = Counter()
cxy = Counter()

for text in texts_tokenized:
    for x in text:
        cx[x] += 1
    
    for x, y in set(map(tuple, map(sorted, combinations(text, 2)))):
        cxy[(x,y)] += 1
        
print(len(cx))
print('\nMost common: ', cx.most_common()[:20])
print('\nLeast common: ', cx.most_common()[(len(cx)-20):])

99044

Most common:  [('hn', 20237), ('show', 10753), ('new', 10080), ('ask', 9582), ('data', 6628), ('google', 5532), ('app', 5124), ('using', 4613), ('us', 4189), ('web', 4134), ('startup', 3849), ('open', 3828), ('first', 3730), ('code', 3705), ('apple', 3695), ('pdf', 3659), ('software', 3558), ('video', 3462), ('tech', 3410), ('free', 3180)]

Least common:  [('codenewbie', 1), ('makefileinspired', 1), ('managerbootstrapper', 1), ('reduxrouting', 1), ('libraryagnostic', 1), ('mocktheclock', 1), ('uncompromising', 1), ('appypaper', 1), ('ringcx', 1), ('getawesomeness', 1), ('keck', 1), ('developerfounderceo', 1), ('integeration', 1), ('gayford', 1), ('superweed', 1), ('prewwii', 1), ('microserivces', 1), ('interdependency', 1), ('tempted', 1), ('isare', 1)]


In [34]:
# Remove infrequent unigrams.

# Terms that appear less frequently than a certain threshold are removed from consideration to help reduce the dimensionality of the data and focus the analysis on more significant terms.
print('%d tokens before' % len(cx))

min_count = (1 / 2000) * len(df) # = 146.5595

for x in list(cx.keys()):
    if cx[x] < min_count:
        del cx[x]

# Remove infrequent bigrams.

for x, y in list(cxy.keys()):
    if x not in cx or y not in cx:
        del cxy[(x, y)]

print('%d tokens after' % len(cx))
print('\nMost common:', cx.most_common()[:20])
print('\nLeast common:', cx.most_common()[(len(cx)-20):])

99044 tokens before
2022 tokens after

Most common: [('hn', 20237), ('show', 10753), ('new', 10080), ('ask', 9582), ('data', 6628), ('google', 5532), ('app', 5124), ('using', 4613), ('us', 4189), ('web', 4134), ('startup', 3849), ('open', 3828), ('first', 3730), ('code', 3705), ('apple', 3695), ('pdf', 3659), ('software', 3558), ('video', 3462), ('tech', 3410), ('free', 3180)]

Least common: [('views', 148), ('emulator', 148), ('directory', 148), ('director', 148), ('amiga', 148), ('bigger', 147), ('hold', 147), ('depression', 147), ('philosophy', 147), ('parts', 147), ('infographic', 147), ('average', 147), ('scam', 147), ('generating', 147), ('targets', 147), ('volkswagen', 147), ('investor', 147), ('classes', 147), ('match', 147), ('timeline', 147)]


In [35]:
# Build unigram <-> index lookup.

# Lookup tables (x2i and i2x) are created to map words to indices and vice versa. This is useful for matrix operations where words need to be represented as numerical indices.
x2i, i2x = {}, {}
for i, x in enumerate(cx.keys()):
    x2i[x] = i
    i2x[i] = x
    
# Sum unigram and bigram counts for computing probabilities.
# i.e. p(x) = count(x) / sum(all counts).

sx = sum(cx.values())
sxy = sum(cxy.values())

# Pointwise Mutual Information (PMI)

Pointwise Mutual Information (PMI) is a statistical measure to calculate the association between two words in a given corpus. PMI is calculated by comparing the probability of the co-occurrence of two words with their individual probabilities of occurrence.

The formula for PMI is as follows:

$$
\text{PMI}(x,y) = \log\left(\frac{P(x,y)}{P(x)P(y)}\right)
$$

where:

- \(x\) and \(y\) are two words being considered,
- \(P(x)\) is the probability of the occurrence of word \(x\) in the corpus,
- \(P(y)\) is the probability of the occurrence of word \(y\) in the corpus, and
- \(P(x,y)\) is the probability of the co-occurrence of words \(x\) and \(y\) in the corpus.

For terms with three words, the formula becomes:

$$
\text{PMI}(z, y, x) = \log\left(\frac{P(z,y,x)}{P(z)P(y)P(x)}\right)
$$

$$
= \log\left(\frac{P(z|y, x)P(y|x)P(x)}{P(z)P(y)P(x)}\right)
$$

$$
= \log\left(\frac{P(z|y, x)P(y|x)}{P(z)P(y)}\right)
$$

PMI values can range from \(-\infty\) to \(\infty\). Positive PMI values indicate that the words have a strong association, while negative values indicate that the words are unlikely to appear together.

## Example

Consider a small corpus of text:

> "The cat sat on the mat. The dog sat on the mat."

Let’s calculate the PMI of the words "cat" and "mat":

- P(cat) = 1/6    
- P(mat) = 2/6 = 1/3    
- P(cat, mat) = 1/6    

$$
\text{PMI}(\text{cat, mat}) = \log\left(\frac{\frac{1}{6}}{\frac{1}{6} \cdot \frac{1}{3}}\right) = \log(3) \approx 1.0986
$$

The positive PMI value suggests that "cat" and "mat" have a strong association and are likely to appear together.

## Real-time Application Considerations

In real-time problems, calculating PMI for longer terms (length > 2) is still very costly for any relatively large corpus of text. We can either go for calculating it only for a two-word term or choose to skip it if we know that there are only a few occurrences of such terms.

PMI is commonly used in various NLP tasks such as information retrieval, topic modeling, and sentiment analysis, to identify and analyze the relationships between words in a corpus.


Reference- [Pointwise Mutual Information in NLP](https://wisdomml.in/concept-of-pointwise-mutual-information-in-nlp/)

In [36]:
# Accumulate data, rows, and cols to build sparse PMI matrix

# The probabilities of each unigram and bigram are calculated based on their frequencies. These probabilities are used to compute the PMI scores for each word pair, which are then stored in a sparse matrix format to optimize memory usage.
from scipy.sparse import csc_matrix
from pprint import pformat

pmi_samples = Counter()
data, rows, cols = [], [], []

for (x, y), n in cxy.items():
    rows.append(x2i[x])
    cols.append(x2i[y])
    data.append(log((n / sxy) / (cx[x] / sx) / (cx[y] / sx)))
    pmi_samples[(x, y)] = data[-1]

# A sparse matrix is constructed using the computed PMI values. This matrix represents the pointwise mutual information between all pairs of words in the dataset.
PMI = csc_matrix((data, (rows, cols)))

print('%d non-zero elements' % PMI.count_nonzero())
print('\nSample PMI values\n', pformat(pmi_samples.most_common()[:15]))

559498 non-zero elements

Sample PMI values
 [(('cheat', 'sheet'), 7.741703919492517),
 (('gravitational', 'waves'), 7.594274561632172),
 (('peter', 'thiel'), 7.501114832399731),
 (('oculus', 'rift'), 7.444931552717988),
 (('nobel', 'prize'), 7.430011693174276),
 (('cook', 'tim'), 7.387140034347794),
 (('virus', 'zika'), 7.218029192769757),
 (('edward', 'snowden'), 7.154950416854075),
 (('clinton', 'hillary'), 7.109859339780078),
 (('area', 'bay'), 7.103617893940743),
 (('boot', 'spring'), 7.096688700497847),
 (('states', 'united'), 7.069464109429888),
 (('korea', 'north'), 7.054418293068599),
 (('panama', 'papers'), 7.0542136375644935),
 (('elon', 'musk'), 6.952284458229978)]


# Singular Value Decomposition (SVD)

Singular Value Decomposition is a way to factor a matrix \( A \) into three matrices, as follows:

$$ A = U \cdot S \cdot V^T $$

Where \( U \) and \( V \) are orthogonal matrices, and \( S \) is a diagonal matrix containing the singular values of \( A \).

## Note:

- The matrix is considered an **orthogonal matrix** if the product of a matrix and its transpose gives an identity value.
- A matrix is **diagonal** if it has non-zero elements only in the diagonal, running from the upper left to the lower right corner of the matrix.

Here, \( U \) and \( V \) represent the left and right singular vectors of \( A \), respectively, and \( S \) represents the singular values of \( A \).

## Algorithm for Computing SVD

The algorithm for computing the SVD of matrix \( A \) can be summarized in the following steps:

1. Compute the eigendecomposition of the symmetric matrix \( A^T A \). This can be done using any standard eigendecomposition algorithm.
2. Compute the singular values of \( A \) as the square root of the eigenvalues of \( A^T A \). Sort the singular values in descending order.
3. Compute the left and right singular vectors of \( A \) as follows:
   - For each singular value, find the corresponding eigenvector of \( A^T A \).
   - Normalize each eigenvector to have a unit length.
   - The left singular vectors of \( A \) are the eigenvectors of \( AA^T \) corresponding to the nonzero singular values of \( A \).
   - The right singular vectors of \( A \) are the normalized eigenvectors of \( A^T A \).

4. Assemble the SVD of \( A \) as follows:
   - The diagonal entries of \( S \) are the singular values of \( A \), sorted in descending order.
   - The columns of \( U \) are the corresponding left singular vectors of \( A \).
   - The columns of \( V \) are the corresponding right singular vectors of \( A \).

## Application in PMI Matrix Factorization and Recommender Systems

The PMI matrix is factorized using Singular Value Decomposition (SVD), which decomposes the matrix into three smaller matrices. This step reduces the dimensionality of the data and uncovers latent semantic structures within it.

SVD is applied to the PMI matrix, specifying the number of dimensions (k) to reduce to. The resulting matrix \( U \) contains vectors representing each word in a reduced-dimensional space.

SVD breaks down a large, complex matrix (which could represent, for example, user ratings for different movies) into simpler, smaller pieces that capture the most important underlying patterns. This can reveal hidden relationships between items in the dataset, like uncovering latent genres in movies that aren't explicitly labeled but are reflected in user ratings (e.g., a "dark comedy" genre that's not officially recognized but consistently liked by a certain group of users).

### Benefits of Using SVD in Recommender Systems

1. **Dimensionality Reduction**: It simplifies the data by reducing the number of variables under consideration, focusing only on those that contribute most significantly to the variation in the dataset. This makes the system more efficient and less prone to overfitting.
2. **Feature Extraction**: It identifies latent features that represent underlying patterns in the data, such as hidden movie genres or unspoken user preferences. These features can then be used to make more accurate recommendations by matching items and users based on these deeper, hidden characteristics.

By applying PMI, we can find strong word associations to understand content relationships better, and with SVD, we can uncover and utilize latent features in the data to make our recommender system more insightful and effective.


In [42]:
# Factorize the PMI matrix using sparse SVD

from scipy.sparse.linalg import svds

U, S, V = svds(PMI, k=20) 
norms = np.sqrt(np.sum(np.square(U), axis=1, keepdims=True))
U /= np.maximum(norms, 1e-7)

In [41]:
# Show some nearest neighbor samples

k = 5

word = input('Enter search word: ')
nearest_neighbours = {}

# by computing the cosine similarity between the search word's vector and all other word vectors, we find and display the nearest neighbors to this word based on their vector representations in the reduced-dimensional space
for x in cx:
    if x == word:
        dd = np.dot(U, U[x2i[x]])
    
        for i in np.argpartition(-1 * dd, k + 1)[:k + 1]:
            if i2x[i] == x: 
                continue
                        
            nearest_neighbours[i2x[i]] = dd[i]

# For each of the nearest neighbor words, we retrieve and display articles from the original dataset that contain the neighbor word, sorted by a scoring metric (num_points).
for nn in nearest_neighbours:
    print(nn)
    similar_articles = df[df['title'].str.contains(nn)]
    display(similar_articles.sort_values(by='num_points',ascending=False)[:5])

Enter search word: javascript
library


,id,title,url,num_points,num_comments,author,created_at
243976,10532957,TensorFlow: open-source library for machine in...,http://tensorflow.org/,1559,196,jmomarty,11/9/2015 13:50
95612,11735393,Libui: GUI library in C,https://github.com/andlabs/libui,373,182,avitex,5/20/2016 3:29
175124,11076390,Bokeh a Python interactive visualization library,http://bokeh.pydata.org/en/latest/,295,66,sonabinu,2/10/2016 21:56
58224,12064022,"Rustls: new, modern TLS library written in Rust",https://github.com/ctz/rustls,287,108,adamnemecek,7/10/2016 0:35
178110,11053525,Records: Python library for making raw SQL que...,https://github.com/kennethreitz/records,257,129,infinite8s,2/7/2016 16:53


es6


,id,title,url,num_points,num_comments,author,created_at
42450,12201387,Pitfalls of fat arrow functions of es6,http://quanz.in/why-dont-i-like-fat-arrow-from...,7,0,rupoJS,8/1/2016 10:48
40400,12218304,"Free Modern JavaScript Course (webpack, nodejs...",http://courses.angularclass.com/courses/modern...,4,1,gdi2290,8/3/2016 14:51
115106,11568631,Show HN: RunJS-SlackBot-A Slack bot that runs ...,https://github.com/prashantagarwal/RunJS-Slackbot,2,0,titanprashant,4/26/2016 0:52
162891,11172862,Best practice of building chrome extension usi...,https://github.com/ProjecToday/es6-chrome-exte...,2,0,timqian,2/25/2016 7:14
236116,10591583,Jump.js a 0 dependency smooth scrolling es6 l...,https://github.com/callmecavs/jump.js,2,1,jaxgeller,11/18/2015 23:42


js


,id,title,url,num_points,num_comments,author,created_at
199984,10882563,TrendMicro Node.js HTTP server listening on lo...,https://code.google.com/p/google-security-rese...,1030,229,tptacek,1/11/2016 19:22
114390,11574705,Node.js v6.0 Released,https://nodejs.org/en/blog/announcements/v6-re...,728,159,JoshGlazebrook,4/26/2016 19:14
158811,11204481,Free React.js Fundamentals Course,http://courses.reactjsprogram.com/courses/reac...,676,112,tm33,3/1/2016 17:47
26393,12338365,Node.js is one of the worst things to happen t...,http://harmful.cat-v.org/software/node.js,624,552,behnamoh,8/22/2016 18:33
165432,11153757,Draft.js Rich Text Editor Framework for React,http://facebook.github.io/draft-js/,592,112,tilt,2/22/2016 19:57


java


,id,title,url,num_points,num_comments,author,created_at
223060,10692247,java.nio.file.WatchService is subtly broken on...,http://blog.omega-prime.co.uk/?p=161,85,6,lelf,12/7/2015 20:03
30424,12302609,Surfingkeys Expand your browser with javascri...,https://github.com/brookhong/Surfingkeys,77,26,ldong,8/17/2016 4:55
112674,11588927,Deprecating: java.util.Optional.get()?,http://royvanrijn.com/blog/2016/04/deprecating...,53,90,redcodenl,4/28/2016 14:05
224003,10685137,Show HN: Kipplr The most absurd application i...,http://www.kipplr.xyz,18,2,gioscarab,12/6/2015 12:49
149824,11276940,"Show HN: Rogue AI Dungeon, javacript bot scrip...",http://bovard.github.io/raid/,17,4,cdubzzz,3/13/2016 9:16


json


,id,title,url,num_points,num_comments,author,created_at
123721,11497826,"Hjson, the Human JSON",http://hjson.org/,257,214,56k,4/14/2016 16:06
143684,11328102,New JSON parser for Go: up to 9x faster than `...,https://github.com/buger/jsonparser,20,14,LeonidBugaev,3/21/2016 14:05
33633,12274980,Ask HN: Is anyone using json schema?,NaN,14,17,luney,8/12/2016 12:17
183140,11012770,Picky.json Dump JSON (raw or URL). Click Some...,http://pickyjson.com,14,0,CorySimmons,2/1/2016 16:32
177640,11058070,Show HN: Electron-json-storage Easily write a...,https://github.com/jviotti/electron-json-storage,12,0,jviotti,2/8/2016 13:57
